In [1]:
import numpy as np
import pandas as pd
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import re
import regex
import datetime
from os import path

%matplotlib inline

In [57]:
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [58]:
def startWithDTandroid(s):
    # pattern for date and time
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -' 
    result = re.match(pattern, s)
    if result:
        return True
    return False

In [59]:
def FindAuthor(s):
    s=s.split(":")  # message starts after authors name and :
    if len(s)==2:  # date, time then author should be 2
        return True
    else:
        return False

In [60]:
#datapoint for android devices
def getDataPointAndroid(line):   
    splitLine = line.split(' - ') #date time - Author : message
    dateTime = splitLine[0]  #so when we split by - we get date time at 0 index
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])  #message will be from ":" till message ends
    if FindAuthor(message): 
        splitMessage = message.split(':') 
        author = splitMessage[0] 
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    return date, time, author, message

In [61]:
# datapoint for ios devices
def getDataPointios(line):
    splitLine = line.split('] ')
    dateTime = splitLine[0]
    if ',' in dateTime:
        date, time = dateTime.split(',')
    else:
        date, time = dateTime.split(' ')
    message = ' '.join(splitLine[1:])
    if FindAuthor(message):
        splitMessage = message.split(':')
        author = splitMessage[0]
        message = ' '.join(splitMessage[1:])
    else:
        author = None
    if time[5]==":":
        time = time[:5]+time[-3:]
    else:
        if 'AM' in time or 'PM' in time:
            time = time[:6]+time[-3:]
        else:
            time = time[:6]
    return date, time, author, message

In [62]:
# emojis
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return emoji_list



In [63]:
# data parser
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
conversationPath = '../WhatsApp.txt' 
with open(conversationPath, encoding="utf-8") as fp:
    device=''
    first=fp.readline()
    print(first)
    if '[' in first:
        device='ios'
    else:
        device="android"
    fp.readline() 
    messageBuffer = [] 
    date, time, author = None, None, None #message contains 4 things-"date","time","author","message" 
    while True:
        line = fp.readline() 
        if not line: 
            break
        if device=="ios":
            line = line.strip()
            if startWithDTios(line):
                if len(messageBuffer) > 0:
                    parsedData.append([date, time, author, ' '.join(messageBuffer)])
                messageBuffer.clear()
                date, time, author, message = getDataPointios(line)
                messageBuffer.append(message)
            else:
                line= (line.encode('ascii', 'ignore')).decode("utf-8")
                if startsWithDateAndTimeios(line):
                    if len(messageBuffer) > 0:
                        parsedData.append([date, time, author, ' '.join(messageBuffer)])
                    messageBuffer.clear()
                    date, time, author, message = getDataPointios(line)
                    messageBuffer.append(message)
                else:
                    messageBuffer.append(line)
        else:
            line = line.strip()
            if startWithDTandroid(line): 
                if len(messageBuffer) > 0:
                    parsedData.append([date, time, author, ' '.join(messageBuffer)])
                messageBuffer.clear()
                date, time, author, message = getDataPointAndroid(line)
                messageBuffer.append(message)
            else:
                messageBuffer.append(line)

16/09/2022, 10:18 am - Messages and calls are end-to-end encrypted. No one outside of this chat, not even WhatsApp, can read or listen to them. Tap to learn more.



In [64]:
print(parsedData)

[['16/09/2022', '10:18 am', None, "You joined using this group's invite link"], ['16/09/2022', '10:18 am', '+233 50 228 4018', ' Thank you'], ['16/09/2022', '10:19 am', None, '+27 63 913 2550: https://youtu.be/X1yJbjcbtIQ'], ['16/09/2022', '10:20 am', None, '+27 63 913 2550: https://youtu.be/X1yJbjcbtIQ'], ['16/09/2022', '10:21 am', '+27 63 913 2550', ' Hey the meeting was recorded & is available on You Tube'], ['16/09/2022', '10:23 am', None, "+234 808 142 0124 joined using this group's invite link"], ['16/09/2022', '10:25 am', None, "+254 716 665120 joined using this group's invite link"], ['16/09/2022', '10:27 am', None, "+233 55 806 3904 joined using this group's invite link"], ['16/09/2022', '10:32 am', None, "+234 803 600 8276 joined using this group's invite link"], ['16/09/2022', '10:32 am', '+234 803 600 8276', ' Hi, My name is Sandra, good to be here'], ['16/09/2022', '10:36 am', None, "+234 703 914 5179 joined using this group's invite link"], ['16/09/2022', '10:37 am', None

In [65]:
if device =='android':
        df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message'])
        df["Date"] = pd.to_datetime(df["Date"])
        #df["emoji"] = df["Message"].apply(split_count)
        URLPATTERN = r'(https?://\S+)'
        df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
else:
    df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
    df["Date"] = df["Date"].apply(dateconv)
    df["Date"] = pd.to_datetime(df["Date"],format='%Y-%m-%d')
   # df["emoji"] = df["Message"].apply(split_count)
    URLPATTERN = r'(https?://\S+)'
    df['urlcount'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
    

/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '16/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '17/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '18/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/usr/local/lib/python3.9/dist-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '19/09/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime

In [66]:
df.sample(30)

,Date,Time,Author,Message,urlcount
12991,2022-11-10,2:43 am,+234 907 098 2153,<Media omitted>,0
350,2022-09-16,5:35 pm,+234 903 698 8199,<Media omitted>,0
6947,2022-09-27,11:36 am,+234 803 297 2030,Just check your *My planning* for live learni...,0
8819,2022-09-30,3:18 am,+234 806 735 4033,*git add .*,0
10497,2022-05-10,10:50 am,+234 705 464 0258,<Media omitted> Ⓒ𝐕𝐥𝐤𝐲𝐫𝐞: is a whatsapp userbo...,0
7869,2022-09-28,1:01 pm,+234 817 524 9526,Yes,0
10751,2022-05-10,5:49 pm,+234 703 340 7608,Task 15,0
10618,2022-05-10,4:16 pm,+251 94 661 9484,Yes how?,0
12522,2022-10-10,1:18 pm,+234 703 453 1814,git push is the last command,0
6213,2022-09-26,1:00 am,+234 808 207 9883,Guys anyone still available?,0


In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19137 entries, 0 to 19136
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      19137 non-null  datetime64[ns]
 1   Time      19137 non-null  object        
 2   Author    17664 non-null  object        
 3   Message   19137 non-null  object        
 4   urlcount  19137 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 747.7+ KB


In [68]:
#separating media and text messages

link_messages= df[df['urlcount']>0]
deleted_messages=df[(df["Message"] == " You deleted this message")| (df["Message"] == " This message was deleted.")]
media_messages_df = df[(df['Message'] == ' <Media omitted>')|(df['Message'] == ' image omitted')|(df['Message'] == ' video omitted')|(df['Message'] == ' sticker omitted')]
messages_df = df.drop(media_messages_df.index)
messages_df = messages_df.drop(deleted_messages.index)
messages_df = messages_df.drop(link_messages.index)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19137 entries, 0 to 19136
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      19137 non-null  datetime64[ns]
 1   Time      19137 non-null  object        
 2   Author    17664 non-null  object        
 3   Message   19137 non-null  object        
 4   urlcount  19137 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 747.7+ KB


In [70]:
#creating coliumns for word count and letter count

messages_df['Letter_Count'] = messages_df['Message'].apply(lambda s : len(s))
messages_df['Word_Count'] = messages_df['Message'].apply(lambda s : len(s.split(' '))) #word is calculated by spliting sentence by whitespace
messages_df["MessageCount"]=1

In [71]:
messages_df.head()

,Date,Time,Author,Message,urlcount,Letter_Count,Word_Count,MessageCount
0,2022-09-16,10:18 am,None,You joined using this group's invite link,0,41,7,1
1,2022-09-16,10:18 am,+233 50 228 4018,Thank you,0,10,3,1
4,2022-09-16,10:21 am,+27 63 913 2550,Hey the meeting was recorded & is available o...,0,56,12,1
5,2022-09-16,10:23 am,None,+234 808 142 0124 joined using this group's in...,0,55,10,1
6,2022-09-16,10:25 am,None,+254 716 665120 joined using this group's invi...,0,53,9,1


In [72]:
messages_df.to_csv('../messages_df.csv', index=False)

In [2]:
df = pd.read_csv('messages_df.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16069 entries, 0 to 16068
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          16069 non-null  object
 1   Time          16069 non-null  object
 2   Author        15158 non-null  object
 3   Message       16069 non-null  object
 4   urlcount      16069 non-null  int64 
 5   Letter_Count  16069 non-null  int64 
 6   Word_Count    16069 non-null  int64 
 7   MessageCount  16069 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 1004.4+ KB


In [4]:
df.isna().sum()

Date              0
Time              0
Author          911
Message           0
urlcount          0
Letter_Count      0
Word_Count        0
MessageCount      0
dtype: int64

In [5]:
df.dropna(inplace=True)

In [6]:
df

,Date,Time,Author,Message,urlcount,Letter_Count,Word_Count,MessageCount
1,2022-09-16,10:18 am,+233 50 228 4018,Thank you,0,10,3,1
2,2022-09-16,10:21 am,+27 63 913 2550,Hey the meeting was recorded & is available o...,0,56,12,1
7,2022-09-16,10:32 am,+234 803 600 8276,"Hi, My name is Sandra, good to be here",0,39,10,1
10,2022-09-16,10:38 am,+233 50 228 4018,"Hi, My name is Bernard and yeah am god to be ...",0,50,13,1
11,2022-09-16,10:40 am,+234 907 993 5418,"Hi everyone, Good morning.",0,27,5,1
...,...,...,...,...,...,...,...,...
16064,2022-11-15,9:02 am,+234 815 857 3822,"Please, @NonsoAwaighi please I sent you a req...",0,64,11,1
16065,2022-11-15,9:34 am,+234 809 093 7741,Can i dm,0,9,4,1
16066,2022-11-15,9:39 am,+234 901 467 8606,You can DM let talk how you want it to be like,0,47,13,1
16067,2022-11-15,9:48 am,+234 901 467 8606,First start with finding a partner because th...,0,124,25,1


In [7]:
df['Time'] = pd.to_datetime(df['Time'], format='%I:%M %p').dt.strftime('%H:%M')

In [14]:
df['DateTime'] = pd.to_datetime(df['Date'] + df['Time'])

TypeError: cannot add DatetimeArray and DatetimeArray

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15158 entries, 1 to 16068
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          15158 non-null  datetime64[ns]
 1   Time          15158 non-null  datetime64[ns]
 2   Author        15158 non-null  object        
 3   Message       15158 non-null  object        
 4   urlcount      15158 non-null  int64         
 5   Letter_Count  15158 non-null  int64         
 6   Word_Count    15158 non-null  int64         
 7   MessageCount  15158 non-null  int64         
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 1.0+ MB


In [ ]:
def user_stats(user):
    frnds = messages_df.Author.unique()
    req_df= messages_df[messages_df["Author"] == frnds[user]]
    print(f'Stats of {frnds[user]} -')
    print('Messages Sent', req_df.shape[0])
    words_per_message = (np.sum(req_df['Word_Count']))/req_df.shape[0]
    print('Words per message', words_per_message)
    media = media_messages_df[media_messages_df['Author'] == frnds[i]].shape[0]
    print('Media Messages Sent', media)
    links = sum(link_messages[link_messages['Author'] == frnds[i]]["urlcount"])   
    print('Links Sent', links)
    return
print(user_stats('+27 63 913 2550'))
    
    

IndexError: ignored

In [ ]:
frnds = messages_df.Author.unique()
print(frnds)


[None '+233 50 228 4018' '+27 63 913 2550' '+234 803 600 8276'
 '+234 907 993 5418' '+234 808 142 0124' '+234 902 038 9938'
 '+234 817 923 7491' '+234 817 684 3723' '+234 701 282 3893'
 '+234 913 302 8000' '+234 816 168 0146' '+233 26 312 8361'
 '+27 71 763 3509' '+27 67 283 2576' '+234 816 540 7995'
 '+234 703 344 4352' '+250 783 142 942' '+234 814 818 5691'
 '+250 786 144 128' '+234 916 143 7539' '+221 76 012 60 41'
 '+234 816 557 2936' '+234 810 393 6568' '+234 907 525 5459'
 '+234 814 346 9301' '+234 818 244 8166' '+234 704 286 0289'
 '+234 816 322 8329' '+234 701 399 5569' '+234 915 055 3719'
 '+234 813 498 6796' '+254 710 445224' '+234 813 602 0773'
 '+234 901 433 4521' '+234 907 251 8753' '+234 816 259 7910'
 '+234 708 143 2487' '+234 703 098 2449' '+234 812 940 9637'
 '+234 803 499 3892' '+234 803 623 4870' '+234 901 760 9776'
 '+233 54 329 3380' '+234 903 043 8296' '+234 703 117 0466'
 '+233 24 964 9787' '+249 99 684 6568' '+234 808 526 2847'
 '+234 907 578 9680' '+234 906 565